In [1]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5

In [ ]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

wandb.login(key="Your-api-key")
Dataset_name = "DD"

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


# *Baseline*

In [6]:
import os
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)
    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        return F.log_softmax(x, dim=1)

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)

def run_seed(seed):
    set_seed(seed)
    
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name)

    # Filter out None graphs and create x if missing
    clean_dataset = []
    for data in dataset:
        if data is None:
            continue
        if data.x is None:
            data.x = torch.ones((data.num_nodes, 1), dtype=torch.float)
        clean_dataset.append(data)

    if len(clean_dataset) == 0:
        raise RuntimeError(f"No valid graphs in dataset. Check your {Dataset_name} installation.")

    dataset = clean_dataset

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    in_channels = dataset[0].x.shape[1]
    out_channels = max([data.y.max().item() for data in dataset]) + 1
    model = GCN(in_channels, 64, out_channels).to(device)

    # Shuffle using random.shuffle instead of dataset.shuffle()
    random.shuffle(dataset)

    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_baseline_seed_{seed}",
        group=f"GCN_{Dataset_name}_baseline",
        job_type="baseline",
        reinit=True,
        config={
            "model":"GCN",
            "dataset":Dataset_name,
            "hidden_channels":64,
            "lr":0.001,
            "epochs":EPOCHS,
            "task":"Baseline",
            "seed":seed
        }
    )

    def train():
        model.train()
        total_loss = 0.0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index, batch.batch)
            loss = F.nll_loss(out, batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        return total_loss / len(train_loader)

    def test(loader):
        model.eval()
        correct = 0
        for batch in loader:
            batch = batch.to(device)
            out = model(batch.x, batch.edge_index, batch.batch)
            pred = out.argmax(dim=1)
            correct += (pred == batch.y).sum().item()
        return correct / len(loader.dataset)

    for epoch in range(1, EPOCHS + 1):
        loss = train()
        test_acc = test(test_loader)
        train_acc = test(train_loader)
        
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    for s in range(1, 31):
        run_seed(s)


Processing...
Done!


Seed 1 | Epoch 001 | Loss: 0.6838 | Test Acc: 0.5720
Seed 1 | Epoch 002 | Loss: 0.6744 | Test Acc: 0.5720
Seed 1 | Epoch 003 | Loss: 0.6723 | Test Acc: 0.5720
Seed 1 | Epoch 004 | Loss: 0.6678 | Test Acc: 0.5720
Seed 1 | Epoch 005 | Loss: 0.6670 | Test Acc: 0.5847
Seed 1 | Epoch 006 | Loss: 0.6534 | Test Acc: 0.6441
Seed 1 | Epoch 007 | Loss: 0.6446 | Test Acc: 0.6144
Seed 1 | Epoch 008 | Loss: 0.6232 | Test Acc: 0.6144
Seed 1 | Epoch 009 | Loss: 0.6066 | Test Acc: 0.6610
Seed 1 | Epoch 010 | Loss: 0.5952 | Test Acc: 0.6610
Seed 1 | Epoch 011 | Loss: 0.6002 | Test Acc: 0.6568
Seed 1 | Epoch 012 | Loss: 0.5907 | Test Acc: 0.6610
Seed 1 | Epoch 013 | Loss: 0.5831 | Test Acc: 0.6653
Seed 1 | Epoch 014 | Loss: 0.5769 | Test Acc: 0.6653
Seed 1 | Epoch 015 | Loss: 0.5767 | Test Acc: 0.6229
Seed 1 | Epoch 016 | Loss: 0.5770 | Test Acc: 0.6864
Seed 1 | Epoch 017 | Loss: 0.5678 | Test Acc: 0.6568
Seed 1 | Epoch 018 | Loss: 0.5602 | Test Acc: 0.6483
Seed 1 | Epoch 019 | Loss: 0.5742 | Test Acc: 

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▆▅▅▄▄▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▃▂▂▂▂▂▁▂▂▁▁▂▁▁▁
test_acc,▁▁▁▁▂▃▃▆▆▆▆▆▄▇▆▆▆▇▆▇▆▆▇▇▆▆▆▆▆▄▆▇▇▇█▆▅▇▆█
train_acc,▁▁▁▁▂▅▄▆▇▅▇▇▅▇▆▇▇▇▇▇▇▇▇▇▇▇██▅▇█▇████████
epoch,50
loss,0.5268
test_acc,0.70339
train_acc,0.75584


Seed 2 | Epoch 001 | Loss: 0.6861 | Test Acc: 0.5805
Seed 2 | Epoch 002 | Loss: 0.6772 | Test Acc: 0.5805
Seed 2 | Epoch 003 | Loss: 0.6753 | Test Acc: 0.5805
Seed 2 | Epoch 004 | Loss: 0.6698 | Test Acc: 0.5805
Seed 2 | Epoch 005 | Loss: 0.6616 | Test Acc: 0.5805
Seed 2 | Epoch 006 | Loss: 0.6527 | Test Acc: 0.6271
Seed 2 | Epoch 007 | Loss: 0.6408 | Test Acc: 0.6610
Seed 2 | Epoch 008 | Loss: 0.6214 | Test Acc: 0.6271
Seed 2 | Epoch 009 | Loss: 0.6030 | Test Acc: 0.6271
Seed 2 | Epoch 010 | Loss: 0.6049 | Test Acc: 0.6610
Seed 2 | Epoch 011 | Loss: 0.6006 | Test Acc: 0.7076
Seed 2 | Epoch 012 | Loss: 0.5901 | Test Acc: 0.6907
Seed 2 | Epoch 013 | Loss: 0.5928 | Test Acc: 0.7076
Seed 2 | Epoch 014 | Loss: 0.5828 | Test Acc: 0.6737
Seed 2 | Epoch 015 | Loss: 0.5841 | Test Acc: 0.6737
Seed 2 | Epoch 016 | Loss: 0.5860 | Test Acc: 0.6822
Seed 2 | Epoch 017 | Loss: 0.5862 | Test Acc: 0.6568
Seed 2 | Epoch 018 | Loss: 0.5793 | Test Acc: 0.6907
Seed 2 | Epoch 019 | Loss: 0.5763 | Test Acc: 

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,███▇▇▆▅▄▄▄▄▃▃▃▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
test_acc,▁▁▁▁▁▅▃▃▅▇▇▆▆▆▅▇█▇▆▇▆▇▇▇▇▇▆█▆█▇█▇▆▇▇███▇
train_acc,▁▁▁▁▁▅▄▃▆▇▇▇▆▇▅▇▇▇▆█▇▆██▇▇█▆▇▆██▇█▇█████
epoch,50
loss,0.53415
test_acc,0.69068
train_acc,0.73567


...

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█████▇▆▅▄▄▄▄▃▃▃▃▂▃▃▃▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▂▁▁▁▁
test_acc,▁▁▁▁▁▅▆▇▃▅▇▇▇▇▇█▆█▆▆█▇██▆▇▆██▇██▇█▆▇▇▇▇▇
train_acc,▁▁▁▁▁▄▆▅▆▄▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇██▇███▇██
epoch,50
loss,0.54251
test_acc,0.71186
train_acc,0.73885


Seed 29 | Epoch 001 | Loss: 0.6898 | Test Acc: 0.6059
Seed 29 | Epoch 002 | Loss: 0.6778 | Test Acc: 0.6059
Seed 29 | Epoch 003 | Loss: 0.6747 | Test Acc: 0.6059
Seed 29 | Epoch 004 | Loss: 0.6716 | Test Acc: 0.6059
Seed 29 | Epoch 005 | Loss: 0.6668 | Test Acc: 0.6059
Seed 29 | Epoch 006 | Loss: 0.6590 | Test Acc: 0.6059
Seed 29 | Epoch 007 | Loss: 0.6413 | Test Acc: 0.6525
Seed 29 | Epoch 008 | Loss: 0.6291 | Test Acc: 0.6314
Seed 29 | Epoch 009 | Loss: 0.6150 | Test Acc: 0.6610
Seed 29 | Epoch 010 | Loss: 0.5985 | Test Acc: 0.6737
Seed 29 | Epoch 011 | Loss: 0.5962 | Test Acc: 0.6822
Seed 29 | Epoch 012 | Loss: 0.5838 | Test Acc: 0.6610
Seed 29 | Epoch 013 | Loss: 0.5795 | Test Acc: 0.6780
Seed 29 | Epoch 014 | Loss: 0.5702 | Test Acc: 0.6949
Seed 29 | Epoch 015 | Loss: 0.5735 | Test Acc: 0.6695
Seed 29 | Epoch 016 | Loss: 0.5713 | Test Acc: 0.6695
Seed 29 | Epoch 017 | Loss: 0.5776 | Test Acc: 0.6949
Seed 29 | Epoch 018 | Loss: 0.5850 | Test Acc: 0.6695
Seed 29 | Epoch 019 | Loss: 

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▇▅▅▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▁▁▁▁▁▁▁▁▁
test_acc,▁▁▁▁▁▃▅▆▆▅▇▆▆▆▆▇▇▆██▇▆▆█▆▇▅▇▇▆▆▇▇▇▆▆▆█▇▆
train_acc,▁▁▁▂▂▄▆▆▇▆▇▇▇▇▇▇█▇█▇▇█▇█▇█▇▇█▇█▇████▇██▇
epoch,50
loss,0.54299
test_acc,0.67797
train_acc,0.72824


Seed 30 | Epoch 001 | Loss: 0.6840 | Test Acc: 0.5890
Seed 30 | Epoch 002 | Loss: 0.6747 | Test Acc: 0.5890
Seed 30 | Epoch 003 | Loss: 0.6732 | Test Acc: 0.5890
Seed 30 | Epoch 004 | Loss: 0.6692 | Test Acc: 0.5890
Seed 30 | Epoch 005 | Loss: 0.6726 | Test Acc: 0.5890
Seed 30 | Epoch 006 | Loss: 0.6587 | Test Acc: 0.6144
Seed 30 | Epoch 007 | Loss: 0.6483 | Test Acc: 0.7076
Seed 30 | Epoch 008 | Loss: 0.6313 | Test Acc: 0.6864
Seed 30 | Epoch 009 | Loss: 0.6211 | Test Acc: 0.7076
Seed 30 | Epoch 010 | Loss: 0.6033 | Test Acc: 0.6992
Seed 30 | Epoch 011 | Loss: 0.6064 | Test Acc: 0.7119
Seed 30 | Epoch 012 | Loss: 0.5957 | Test Acc: 0.6949
Seed 30 | Epoch 013 | Loss: 0.5963 | Test Acc: 0.6907
Seed 30 | Epoch 014 | Loss: 0.5798 | Test Acc: 0.6907
Seed 30 | Epoch 015 | Loss: 0.5858 | Test Acc: 0.6949
Seed 30 | Epoch 016 | Loss: 0.5891 | Test Acc: 0.6907
Seed 30 | Epoch 017 | Loss: 0.5826 | Test Acc: 0.6907
Seed 30 | Epoch 018 | Loss: 0.5765 | Test Acc: 0.7076
Seed 30 | Epoch 019 | Loss: 

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▅▅▄▄▄▃▃▃▃▂▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁
test_acc,▁▁▁▁▁█▇█▇█▇▇▇▇▇▇▆█▇▆▆▇▇▆▆▆▇▆▇▇▇▆▇▇▆▇▇▇▇█
train_acc,▁▁▁▁▁▇▅▇▆▇▇▇▆▇▆█▇████▇▇▇▇█▇██████▇██████
epoch,50
loss,0.54553
test_acc,0.70763
train_acc,0.73779


# *BFS*

In [ ]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def add_bfs_rank(d):
    d = d.clone()
    if d.x is None:
        d.x = torch.ones((d.num_nodes, 1), dtype=torch.float)
    G = to_networkx(d, to_undirected=True)
    root = int(d.edge_index[0][0].item()) if d.edge_index.numel() > 0 else 0
    bfs_order = list(nx.bfs_tree(G, root)) if G.number_of_nodes() > 0 else []
    bfs_rank = torch.full((d.num_nodes, 1), -1.0, dtype=torch.float)
    L = max(1, len(bfs_order))
    for i, node in enumerate(bfs_order):
        bfs_rank[node] = float(i) / float(L)
    d.x = torch.cat([d.x, bfs_rank], dim=1)
    return d

def run_seed(seed):
    set_seed(seed)
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name, transform=add_bfs_rank)
    torch.manual_seed(seed)
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    in_channels = dataset[0].x.size(1)
    model = GCN(in_channels, 64, dataset.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_BFS_seed_{seed}",
        group=f"GCN_{Dataset_name}_BFS",
        job_type="bfs",
        reinit=True,
        config={"model":"GCN","hidden_channels":64,"epochs":EPOCHS,"lr":0.001,"dataset":"REDDIT-BINARY","task":"BFS","positional_encoding":"BFS","seed":seed}
    )

    def train():
        model.train()
        total_loss = 0.0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        return total_loss / len(train_loader)

    def test(loader):
        model.eval()
        correct = 0
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(loader.dataset)

    for epoch in range(1, EPOCHS + 1):
        loss = train()
        test_acc = test(test_loader)
        train_acc = test(train_loader)
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})
    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


Seed 1 | Epoch 001 | Loss: 0.6887 | Test Acc: 0.5678
Seed 1 | Epoch 002 | Loss: 0.6754 | Test Acc: 0.5678
Seed 1 | Epoch 003 | Loss: 0.6727 | Test Acc: 0.5678
Seed 1 | Epoch 004 | Loss: 0.6702 | Test Acc: 0.5678
Seed 1 | Epoch 005 | Loss: 0.6636 | Test Acc: 0.5720
Seed 1 | Epoch 006 | Loss: 0.6598 | Test Acc: 0.5805
Seed 1 | Epoch 007 | Loss: 0.6436 | Test Acc: 0.6356
Seed 1 | Epoch 008 | Loss: 0.6299 | Test Acc: 0.6525
Seed 1 | Epoch 009 | Loss: 0.6138 | Test Acc: 0.6610
Seed 1 | Epoch 010 | Loss: 0.6040 | Test Acc: 0.6653
Seed 1 | Epoch 011 | Loss: 0.5877 | Test Acc: 0.6653
Seed 1 | Epoch 012 | Loss: 0.5834 | Test Acc: 0.6483
Seed 1 | Epoch 013 | Loss: 0.5764 | Test Acc: 0.6610
Seed 1 | Epoch 014 | Loss: 0.5961 | Test Acc: 0.6907
Seed 1 | Epoch 015 | Loss: 0.5676 | Test Acc: 0.6864
Seed 1 | Epoch 016 | Loss: 0.5691 | Test Acc: 0.6737
Seed 1 | Epoch 017 | Loss: 0.5640 | Test Acc: 0.6907
Seed 1 | Epoch 018 | Loss: 0.5627 | Test Acc: 0.6949
Seed 1 | Epoch 019 | Loss: 0.5614 | Test Acc: 

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▇▆▅▅▄▃▃▄▃▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁▂
test_acc,▁▁▁▁▂▆▆▆▆▅██▇██▇▂▇▄█▇▅▇█▇▇▇▂▆▇▇▆▆▇▆▅███▅
train_acc,▁▁▁▁▁▄▅▆▆▆▆▇▆▇▇▇█▅█▆▇█▇█▇██▆██▇███▇█▆██▇
epoch,50
loss,0.54827
test_acc,0.63136
train_acc,0.73673


Seed 2 | Epoch 001 | Loss: 0.6859 | Test Acc: 0.6144
Seed 2 | Epoch 002 | Loss: 0.6799 | Test Acc: 0.6144
Seed 2 | Epoch 003 | Loss: 0.6774 | Test Acc: 0.6144
Seed 2 | Epoch 004 | Loss: 0.6734 | Test Acc: 0.6144
Seed 2 | Epoch 005 | Loss: 0.6700 | Test Acc: 0.6144
Seed 2 | Epoch 006 | Loss: 0.6661 | Test Acc: 0.6229
Seed 2 | Epoch 007 | Loss: 0.6619 | Test Acc: 0.6186
Seed 2 | Epoch 008 | Loss: 0.6526 | Test Acc: 0.6271
Seed 2 | Epoch 009 | Loss: 0.6475 | Test Acc: 0.6780
Seed 2 | Epoch 010 | Loss: 0.6305 | Test Acc: 0.7034
Seed 2 | Epoch 011 | Loss: 0.6226 | Test Acc: 0.7246
Seed 2 | Epoch 012 | Loss: 0.6114 | Test Acc: 0.6992
Seed 2 | Epoch 013 | Loss: 0.6100 | Test Acc: 0.7161
Seed 2 | Epoch 014 | Loss: 0.6004 | Test Acc: 0.7373
Seed 2 | Epoch 015 | Loss: 0.6016 | Test Acc: 0.7203
Seed 2 | Epoch 016 | Loss: 0.6014 | Test Acc: 0.7373
Seed 2 | Epoch 017 | Loss: 0.5886 | Test Acc: 0.7415
Seed 2 | Epoch 018 | Loss: 0.5912 | Test Acc: 0.7373
Seed 2 | Epoch 019 | Loss: 0.5817 | Test Acc: 

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▆▆▅▅▄▄▄▄▃▃▄▃▃▃▂▃▂▂▃▃▂▂▃▂▂▁▂▁▁▁▁▁▁▁
test_acc,▁▁▁▁▁▁▂▄▅▆▆▇▆▇▇▇▆▇▆▆▇▆▇▇▇▆▅▇▆▇▇▇▆█▆██▇█▇
train_acc,▁▁▁▁▁▁▁▄▆▆▆▇▆▇▇▇▆▇▆▇▇▇▇▇▇▆▅█▇█▇▇▇█▇██▇██
epoch,50
loss,0.54794
test_acc,0.74576
train_acc,0.7569


Seed 3 | Epoch 001 | Loss: 0.6825 | Test Acc: 0.5847
Seed 3 | Epoch 002 | Loss: 0.6773 | Test Acc: 0.5847
Seed 3 | Epoch 003 | Loss: 0.6751 | Test Acc: 0.5847
Seed 3 | Epoch 004 | Loss: 0.6707 | Test Acc: 0.5763
Seed 3 | Epoch 005 | Loss: 0.6659 | Test Acc: 0.5763
Seed 3 | Epoch 006 | Loss: 0.6536 | Test Acc: 0.6059
Seed 3 | Epoch 007 | Loss: 0.6460 | Test Acc: 0.5890
Seed 3 | Epoch 008 | Loss: 0.6362 | Test Acc: 0.6229
Seed 3 | Epoch 009 | Loss: 0.6231 | Test Acc: 0.6949
Seed 3 | Epoch 010 | Loss: 0.6073 | Test Acc: 0.6314
Seed 3 | Epoch 011 | Loss: 0.6004 | Test Acc: 0.6398
Seed 3 | Epoch 012 | Loss: 0.5963 | Test Acc: 0.7076
Seed 3 | Epoch 013 | Loss: 0.5879 | Test Acc: 0.6949
Seed 3 | Epoch 014 | Loss: 0.5934 | Test Acc: 0.7034
Seed 3 | Epoch 015 | Loss: 0.5872 | Test Acc: 0.6949
Seed 3 | Epoch 016 | Loss: 0.5824 | Test Acc: 0.7034
Seed 3 | Epoch 017 | Loss: 0.5789 | Test Acc: 0.7076
Seed 3 | Epoch 018 | Loss: 0.5823 | Test Acc: 0.6822
Seed 3 | Epoch 019 | Loss: 0.5806 | Test Acc: 

...

Seed 24 | Epoch 001 | Loss: 0.6841 | Test Acc: 0.5805
Seed 24 | Epoch 002 | Loss: 0.6794 | Test Acc: 0.5805
Seed 24 | Epoch 003 | Loss: 0.6773 | Test Acc: 0.5805
Seed 24 | Epoch 004 | Loss: 0.6732 | Test Acc: 0.5805
Seed 24 | Epoch 005 | Loss: 0.6727 | Test Acc: 0.5805
Seed 24 | Epoch 006 | Loss: 0.6697 | Test Acc: 0.5805
Seed 24 | Epoch 007 | Loss: 0.6614 | Test Acc: 0.5805
Seed 24 | Epoch 008 | Loss: 0.6508 | Test Acc: 0.6144
Seed 24 | Epoch 009 | Loss: 0.6308 | Test Acc: 0.6907
Seed 24 | Epoch 010 | Loss: 0.6177 | Test Acc: 0.6695
Seed 24 | Epoch 011 | Loss: 0.6118 | Test Acc: 0.6695
Seed 24 | Epoch 012 | Loss: 0.6021 | Test Acc: 0.6949
Seed 24 | Epoch 013 | Loss: 0.5898 | Test Acc: 0.7203
Seed 24 | Epoch 014 | Loss: 0.5975 | Test Acc: 0.6949
Seed 24 | Epoch 015 | Loss: 0.5913 | Test Acc: 0.6907
Seed 24 | Epoch 016 | Loss: 0.5849 | Test Acc: 0.6907
Seed 24 | Epoch 017 | Loss: 0.5785 | Test Acc: 0.6907
Seed 24 | Epoch 018 | Loss: 0.5781 | Test Acc: 0.7119
Seed 24 | Epoch 019 | Loss: 

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█████▇▆▅▅▄▄▄▃▃▃▃▃▃▃▂▃▂▂▃▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▂
test_acc,▁▁▁▁▁▃▆▅▅▆▆▆▆▆▇█▇▆▆▆█▆▇▇▇▇▆▇▇▇▇███▆▇▆██▇
train_acc,▁▁▁▁▁▁▃▆▅▆▅▇▇▇▆▆▇▆▇▇▆▇▆▇▇▇▇▇▇▇███▇▇█▇█▇█
epoch,50
loss,0.55883
test_acc,0.72458
train_acc,0.75902


Seed 25 | Epoch 001 | Loss: 0.6916 | Test Acc: 0.6144
Seed 25 | Epoch 002 | Loss: 0.6802 | Test Acc: 0.6144
Seed 25 | Epoch 003 | Loss: 0.6761 | Test Acc: 0.6144
Seed 25 | Epoch 004 | Loss: 0.6741 | Test Acc: 0.6144
Seed 25 | Epoch 005 | Loss: 0.6698 | Test Acc: 0.6102
Seed 25 | Epoch 006 | Loss: 0.6618 | Test Acc: 0.6271
Seed 25 | Epoch 007 | Loss: 0.6512 | Test Acc: 0.7034
Seed 25 | Epoch 008 | Loss: 0.6365 | Test Acc: 0.6610
Seed 25 | Epoch 009 | Loss: 0.6193 | Test Acc: 0.6864
Seed 25 | Epoch 010 | Loss: 0.6086 | Test Acc: 0.6822
Seed 25 | Epoch 011 | Loss: 0.6056 | Test Acc: 0.6949
Seed 25 | Epoch 012 | Loss: 0.5956 | Test Acc: 0.6907
Seed 25 | Epoch 013 | Loss: 0.5946 | Test Acc: 0.7034
Seed 25 | Epoch 014 | Loss: 0.5899 | Test Acc: 0.7203
Seed 25 | Epoch 015 | Loss: 0.5959 | Test Acc: 0.7034
Seed 25 | Epoch 016 | Loss: 0.5916 | Test Acc: 0.6398
Seed 25 | Epoch 017 | Loss: 0.5904 | Test Acc: 0.7203
Seed 25 | Epoch 018 | Loss: 0.5836 | Test Acc: 0.7203
Seed 25 | Epoch 019 | Loss: 

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▇▅▄▄▄▃▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▃▂▂▂▂▂▁▁▂▁▁▁▁▂▁
test_acc,▁▁▁▁▁▆▄▅▅▆▆▇▆▃▇▇▇▇█▇█▇██▇█▇▆▆▇███▆██▆▇▇▇
train_acc,▁▁▁▁▁▆▅▆▆▆▇▇▅▅▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
epoch,50
loss,0.54842
test_acc,0.7161
train_acc,0.74628


Seed 26 | Epoch 001 | Loss: 0.6799 | Test Acc: 0.5339
Seed 26 | Epoch 002 | Loss: 0.6708 | Test Acc: 0.5339
Seed 26 | Epoch 003 | Loss: 0.6675 | Test Acc: 0.5339
Seed 26 | Epoch 004 | Loss: 0.6665 | Test Acc: 0.5339
Seed 26 | Epoch 005 | Loss: 0.6619 | Test Acc: 0.5381
Seed 26 | Epoch 006 | Loss: 0.6491 | Test Acc: 0.5508
Seed 26 | Epoch 007 | Loss: 0.6355 | Test Acc: 0.5975
Seed 26 | Epoch 008 | Loss: 0.6265 | Test Acc: 0.6017
Seed 26 | Epoch 009 | Loss: 0.6140 | Test Acc: 0.6229
Seed 26 | Epoch 010 | Loss: 0.6055 | Test Acc: 0.6822
Seed 26 | Epoch 011 | Loss: 0.5989 | Test Acc: 0.6695
Seed 26 | Epoch 012 | Loss: 0.5942 | Test Acc: 0.6907
Seed 26 | Epoch 013 | Loss: 0.5971 | Test Acc: 0.6737
Seed 26 | Epoch 014 | Loss: 0.5882 | Test Acc: 0.6695
Seed 26 | Epoch 015 | Loss: 0.5798 | Test Acc: 0.6653
Seed 26 | Epoch 016 | Loss: 0.5781 | Test Acc: 0.6822
Seed 26 | Epoch 017 | Loss: 0.5801 | Test Acc: 0.6822
Seed 26 | Epoch 018 | Loss: 0.5754 | Test Acc: 0.6695
Seed 26 | Epoch 019 | Loss: 